In [1]:
from network import deblur
from datagen import flow,image_resize
from config import param_maps
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from helpers import SnapshotCallbackBuilder,TensorBoard,LOSSES,PSNRLoss

/home/anish/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/anish/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/anish/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/anish/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: Futu

In [2]:
MODEL = deblur(upsample=param_maps["upsample_config"],rdb_depth=param_maps["Depth"])
MODEL.load_weights("fas_blur.h5")

In [3]:
def deblur(img):
    """forced to resize on its best working scale,
    training required for large images"""
    read = cv2.imread(img)
    height,width = read.shape[0],read.shape[1]
    if height % 2 != 0:
        height = height-1
    if width % 2 != 0:
        width = width-1

    resized_height,resized_width = height,width
    ima = load_img(img,target_size=(height,width))
    image = ima/np.array(127.5)-1
    image = np.expand_dims(image,axis=0)
    prediction = MODEL.predict(image)
    prediction = prediction+1
    prediction = prediction*127.5
    out = np.abs(prediction).astype(np.uint16)[0]
    out = np.clip(out,0,255)
    cv2.imwrite("outs1.jpg",out)
    return out

def image_resize(image, width = None, height = None, inter = cv2.INTER_AREA):
    dim = None
    (h, w) = image.shape[:2]

    if width is None and height is None:
        return image
    if width is None:
        r = height / float(h)
        dim = (int(w * r), height)

    else:
        r = width / float(w)
        dim = (width, int(h * r))

    resized = cv2.resize(image, dim, interpolation = inter)
    return resized

In [4]:
def predict(image_file):
    image = image_file
    read = cv2.imread(image)
    read = image_resize(read,height = param_maps["scale"])
    cv2.imwrite("outs1.jpg",read)
    deb = deblur("outs1.jpg")

In [9]:
predict("input.jpg")

350
